# Network Pharmacology - Molecular Dynamics Simulation

This notebook performs automated MD simulation using GROMACS.

**Requirements:**
- Docked protein-ligand complex from docking notebook

**Output:**
- RMSD, RMSF, H-bond analysis
- Trajectory visualization

**Duration:** ~50ns (suitable for free Colab, may need multiple sessions)

## 1. Install GROMACS

In [ ]:
%%bash
# Install GROMACS
apt-get update -qq
apt-get install -qq gromacs

# Verify installation
gmx --version | head -5

In [ ]:
# Install Python dependencies
!pip install -q MDAnalysis matplotlib numpy pandas nglview
print("✅ All dependencies installed!")

## 2. Configuration

In [ ]:
# ============================================
# CONFIGURATION - EDIT THIS SECTION
# ============================================

PROJECT_NAME = "mahkota_dewa_dn"

# Simulation parameters
MD_DURATION_NS = 50        # Simulation duration in nanoseconds
TEMPERATURE_K = 310        # Temperature in Kelvin (310K = 37°C)
TIMESTEP_FS = 2            # Timestep in femtoseconds
NSTEPS = int(MD_DURATION_NS * 1e6 / TIMESTEP_FS)  # Total steps

# Equilibration parameters  
NVT_DURATION_PS = 100      # NVT equilibration in picoseconds
NPT_DURATION_PS = 100      # NPT equilibration in picoseconds

# Force field
FORCE_FIELD = "amber99sb-ildn"  # Common for proteins
WATER_MODEL = "tip3p"

print(f"Simulation: {MD_DURATION_NS}ns at {TEMPERATURE_K}K")
print(f"Total steps: {NSTEPS:,}")

## 3. Upload Input Files

In [ ]:
from google.colab import files
import os

# Create directories
for d in ["input", "topol", "em", "nvt", "npt", "md", "analysis"]:
    os.makedirs(d, exist_ok=True)

print("Upload your protein PDB file (prepared, without ligand):")
uploaded_protein = files.upload()
PROTEIN_PDB = list(uploaded_protein.keys())[0]
print(f"✅ Protein: {PROTEIN_PDB}")

print("\nUpload your ligand file (MOL2 or PDB format):")
uploaded_ligand = files.upload()
LIGAND_FILE = list(uploaded_ligand.keys())[0]
print(f"✅ Ligand: {LIGAND_FILE}")

## 4. Prepare Protein Topology

In [ ]:
%%bash -s "$PROTEIN_PDB" "$FORCE_FIELD" "$WATER_MODEL"
# Generate topology for protein
echo "1" | gmx pdb2gmx -f $1 -o topol/protein.gro -p topol/topol.top -i topol/posre.itp -ff $2 -water $3

echo "✅ Protein topology generated!"

## 5. Prepare Ligand Topology (Using ACPYPE)

In [ ]:
# Install ACPYPE for ligand parametrization
!pip install -q acpype

# Generate ligand topology with GAFF force field
!acpype -i {LIGAND_FILE} -b ligand -c bcc -a gaff2

# Move ligand files
!mv ligand.acpype/* topol/

print("✅ Ligand topology generated!")

## 6. Combine Protein and Ligand

In [ ]:
%%bash
# Combine protein and ligand coordinates
cd topol

# Append ligand to protein
grep -h ATOM protein.gro ligand_GMX.gro > complex.gro.tmp

# Fix the header
head -1 protein.gro > complex.gro
natoms=$(grep -c ATOM complex.gro.tmp)
echo " $natoms" >> complex.gro
cat complex.gro.tmp >> complex.gro
tail -1 protein.gro >> complex.gro
rm complex.gro.tmp

# Update topology to include ligand
echo '#include "ligand_GMX.itp"' >> topol.top
echo 'LIG     1' >> topol.top

echo "✅ Complex created!"

## 7. Create Simulation Box and Solvate

In [ ]:
%%bash
cd topol

# Create box (dodecahedron, 1.2nm from solute)
gmx editconf -f complex.gro -o box.gro -c -d 1.2 -bt dodecahedron

# Solvate
gmx solvate -cp box.gro -cs spc216.gro -o solvated.gro -p topol.top

echo "✅ System solvated!"

## 8. Add Ions (Neutralize System)

In [ ]:
# Create ions MDP file
ions_mdp = """
; ions.mdp - used for adding ions
integrator  = steep
emtol       = 1000.0
emstep      = 0.01
nsteps      = 50000
nstlist     = 1
cutoff-scheme = Verlet
ns_type     = grid
coulombtype = cutoff
rcoulomb    = 1.0
rvdw        = 1.0
pbc         = xyz
"""

with open("topol/ions.mdp", "w") as f:
    f.write(ions_mdp)

In [ ]:
%%bash
cd topol

# Prepare for adding ions
gmx grompp -f ions.mdp -c solvated.gro -p topol.top -o ions.tpr -maxwarn 2

# Add ions (neutralize, 0.15M NaCl)
echo "SOL" | gmx genion -s ions.tpr -o system.gro -p topol.top -pname NA -nname CL -neutral -conc 0.15

echo "✅ System neutralized!"

## 9. Energy Minimization

In [ ]:
# Create EM MDP file
em_mdp = """
; em.mdp - energy minimization
integrator  = steep
emtol       = 1000.0
emstep      = 0.01
nsteps      = 50000

nstlist     = 1
cutoff-scheme = Verlet
ns_type     = grid
coulombtype = PME
rcoulomb    = 1.0
rvdw        = 1.0
pbc         = xyz
"""

with open("em/em.mdp", "w") as f:
    f.write(em_mdp)

In [ ]:
%%bash
# Run energy minimization
gmx grompp -f em/em.mdp -c topol/system.gro -p topol/topol.top -o em/em.tpr -maxwarn 2
gmx mdrun -v -deffnm em/em

echo "✅ Energy minimization complete!"

## 10. NVT Equilibration

In [ ]:
# Create NVT MDP file
nvt_mdp = f"""
; nvt.mdp - NVT equilibration
define      = -DPOSRES
integrator  = md
nsteps      = {int(NVT_DURATION_PS * 1000 / TIMESTEP_FS)}
dt          = {TIMESTEP_FS / 1000}

nstxout     = 5000
nstvout     = 5000
nstenergy   = 5000
nstlog      = 5000

continuation = no
constraint_algorithm = lincs
constraints = h-bonds
lincs_iter  = 1
lincs_order = 4

cutoff-scheme = Verlet
ns_type     = grid
nstlist     = 10
rcoulomb    = 1.0
rvdw        = 1.0

coulombtype = PME
pme_order   = 4
fourierspacing = 0.16

tcoupl      = V-rescale
tc-grps     = Protein Non-Protein
tau_t       = 0.1 0.1
ref_t       = {TEMPERATURE_K} {TEMPERATURE_K}

pcoupl      = no
pbc         = xyz
DispCorr    = EnerPres

gen_vel     = yes
gen_temp    = {TEMPERATURE_K}
gen_seed    = -1
"""

with open("nvt/nvt.mdp", "w") as f:
    f.write(nvt_mdp)

In [ ]:
%%bash
# Run NVT equilibration
gmx grompp -f nvt/nvt.mdp -c em/em.gro -r em/em.gro -p topol/topol.top -o nvt/nvt.tpr -maxwarn 2
gmx mdrun -v -deffnm nvt/nvt

echo "✅ NVT equilibration complete!"

## 11. NPT Equilibration

In [ ]:
# Create NPT MDP file
npt_mdp = f"""
; npt.mdp - NPT equilibration
define      = -DPOSRES
integrator  = md
nsteps      = {int(NPT_DURATION_PS * 1000 / TIMESTEP_FS)}
dt          = {TIMESTEP_FS / 1000}

nstxout     = 5000
nstvout     = 5000
nstenergy   = 5000
nstlog      = 5000

continuation = yes
constraint_algorithm = lincs
constraints = h-bonds
lincs_iter  = 1
lincs_order = 4

cutoff-scheme = Verlet
ns_type     = grid
nstlist     = 10
rcoulomb    = 1.0
rvdw        = 1.0

coulombtype = PME
pme_order   = 4
fourierspacing = 0.16

tcoupl      = V-rescale
tc-grps     = Protein Non-Protein
tau_t       = 0.1 0.1
ref_t       = {TEMPERATURE_K} {TEMPERATURE_K}

pcoupl      = Parrinello-Rahman
pcoupltype  = isotropic
tau_p       = 2.0
ref_p       = 1.0
compressibility = 4.5e-5
refcoord_scaling = com

pbc         = xyz
DispCorr    = EnerPres

gen_vel     = no
"""

with open("npt/npt.mdp", "w") as f:
    f.write(npt_mdp)

In [ ]:
%%bash
# Run NPT equilibration
gmx grompp -f npt/npt.mdp -c nvt/nvt.gro -r nvt/nvt.gro -t nvt/nvt.cpt -p topol/topol.top -o npt/npt.tpr -maxwarn 2
gmx mdrun -v -deffnm npt/npt

echo "✅ NPT equilibration complete!"

## 12. Production MD (50ns)

In [ ]:
# Create production MD MDP file
md_mdp = f"""
; md.mdp - Production MD ({MD_DURATION_NS}ns)
integrator  = md
nsteps      = {NSTEPS}
dt          = {TIMESTEP_FS / 1000}

nstxout     = 0
nstvout     = 0
nstxout-compressed = 5000
nstenergy   = 5000
nstlog      = 5000

continuation = yes
constraint_algorithm = lincs
constraints = h-bonds
lincs_iter  = 1
lincs_order = 4

cutoff-scheme = Verlet
ns_type     = grid
nstlist     = 10
rcoulomb    = 1.0
rvdw        = 1.0

coulombtype = PME
pme_order   = 4
fourierspacing = 0.16

tcoupl      = V-rescale
tc-grps     = Protein Non-Protein
tau_t       = 0.1 0.1
ref_t       = {TEMPERATURE_K} {TEMPERATURE_K}

pcoupl      = Parrinello-Rahman
pcoupltype  = isotropic
tau_p       = 2.0
ref_p       = 1.0
compressibility = 4.5e-5

pbc         = xyz
DispCorr    = EnerPres

gen_vel     = no
"""

with open("md/md.mdp", "w") as f:
    f.write(md_mdp)

print(f"Production MD: {MD_DURATION_NS}ns, {NSTEPS:,} steps")

In [ ]:
%%bash
# Prepare production run
gmx grompp -f md/md.mdp -c npt/npt.gro -t npt/npt.cpt -p topol/topol.top -o md/md.tpr -maxwarn 2

echo "✅ Production run prepared!"
echo "Starting {MD_DURATION_NS}ns simulation (this will take several hours)..."

In [ ]:
%%bash
# Run production MD
# Note: This will take several hours. If Colab times out, you can resume with -cpi md/md.cpt
gmx mdrun -deffnm md/md -v

echo "✅ Production MD complete!"

## 13. Analysis - RMSD

In [ ]:
%%bash
# Calculate RMSD
echo "4 4" | gmx rms -s md/md.tpr -f md/md.xtc -o analysis/rmsd.xvg -tu ns

# Calculate RMSD of ligand
echo "13 13" | gmx rms -s md/md.tpr -f md/md.xtc -o analysis/rmsd_ligand.xvg -tu ns

echo "✅ RMSD calculated!"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def parse_xvg(filename):
    """Parse GROMACS XVG file."""
    data = []
    with open(filename, 'r') as f:
        for line in f:
            if not line.startswith(('#', '@')):
                values = [float(x) for x in line.split()]
                data.append(values)
    return np.array(data)

# Plot RMSD
fig, ax = plt.subplots(figsize=(10, 6))

rmsd = parse_xvg('analysis/rmsd.xvg')
ax.plot(rmsd[:, 0], rmsd[:, 1], label='Protein Backbone', color='#2E86AB')

if os.path.exists('analysis/rmsd_ligand.xvg'):
    rmsd_lig = parse_xvg('analysis/rmsd_ligand.xvg')
    ax.plot(rmsd_lig[:, 0], rmsd_lig[:, 1], label='Ligand', color='#A23B72')

ax.set_xlabel('Time (ns)', fontsize=12)
ax.set_ylabel('RMSD (nm)', fontsize=12)
ax.set_title('Root Mean Square Deviation', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('analysis/rmsd_plot.png', dpi=300)
plt.show()

print(f"Average Protein RMSD: {rmsd[:, 1].mean():.3f} nm")
print(f"Std Protein RMSD: {rmsd[:, 1].std():.3f} nm")

## 14. Analysis - RMSF

In [ ]:
%%bash
# Calculate RMSF per residue
echo "4" | gmx rmsf -s md/md.tpr -f md/md.xtc -o analysis/rmsf.xvg -res

echo "✅ RMSF calculated!"

In [ ]:
# Plot RMSF
rmsf = parse_xvg('analysis/rmsf.xvg')

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(rmsf[:, 0], rmsf[:, 1], color='#2E86AB', linewidth=0.8)
ax.fill_between(rmsf[:, 0], 0, rmsf[:, 1], alpha=0.3, color='#2E86AB')

ax.set_xlabel('Residue Number', fontsize=12)
ax.set_ylabel('RMSF (nm)', fontsize=12)
ax.set_title('Root Mean Square Fluctuation per Residue', fontsize=14)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('analysis/rmsf_plot.png', dpi=300)
plt.show()

# Find most flexible residues
top_flex = np.argsort(rmsf[:, 1])[-10:]
print("\nMost flexible residues:")
for i in top_flex[::-1]:
    print(f"  Residue {int(rmsf[i, 0])}: {rmsf[i, 1]:.3f} nm")

## 15. Analysis - Hydrogen Bonds

In [ ]:
%%bash
# Calculate hydrogen bonds between protein and ligand
echo "1 13" | gmx hbond -s md/md.tpr -f md/md.xtc -num analysis/hbond.xvg

echo "✅ H-bond analysis complete!"

In [ ]:
# Plot hydrogen bonds
hbond = parse_xvg('analysis/hbond.xvg')

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(hbond[:, 0] / 1000, hbond[:, 1], color='#F18F01', alpha=0.7)

ax.set_xlabel('Time (ns)', fontsize=12)
ax.set_ylabel('Number of H-bonds', fontsize=12)
ax.set_title('Protein-Ligand Hydrogen Bonds', fontsize=14)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('analysis/hbond_plot.png', dpi=300)
plt.show()

print(f"Average H-bonds: {hbond[:, 1].mean():.1f}")
print(f"Max H-bonds: {hbond[:, 1].max():.0f}")

## 16. Download Results

In [ ]:
import shutil

# Create results package
shutil.make_archive(f"{PROJECT_NAME}_md_results", "zip", "analysis")

# Download
files.download(f"{PROJECT_NAME}_md_results.zip")
print("✅ Results downloaded!")

## Notes

### If Colab Times Out:
You can resume the simulation from the last checkpoint:
```bash
gmx mdrun -deffnm md/md -cpi md/md.cpt -append
```

### Save Checkpoint Regularly:
Download `md/md.cpt` periodically to have a backup.

### MM-PBSA Binding Energy:
For more accurate binding energy calculations, consider using:
- gmx_MMPBSA (requires separate installation)
- g_mmpbsa tool